# Introduction to Python and Bioinformatics

# - The Python Fundamentals

#### Developed by:  A. Fahim, California State Univeristy Long Beach

This notebook is a supplement to the workshop "Introduction to Python and Bioinformatics"


### Next-Gen Sequencing

We will also import the module to process sequences. Do not forget to put
the correct e-mail address.

In [ ]:
from Bio import Entrez, Medline, SeqIO

In [ ]:
Entrez.email = "arjangvt@gmail.com"

In [ ]:
#This gives you the list of available databases
handle = Entrez.einfo()
rec = Entrez.read(handle)
print(rec)

We will now try to find the Cholroquine Resistance Transporter (CRT) gene in
Plasmodium falciparum (the parasite that causes the deadliest form of malaria)
on the nucleotide database:

In [ ]:
handle = Entrez.esearch(db="nucleotide", term='CRT[Gene Name] AND "Plasmodium falciparum"[Organism]')
rec_list = Entrez.read(handle)

if rec_list['RetMax'] < rec_list['Count']:
    handle = Entrez.esearch(db="nucleotide", term='CRT[Gene Name] AND "Plasmodium falciparum"[Organism]',
                            retmax=rec_list['Count'])
    rec_list = Entrez.read(handle)

In [ ]:
id_list = rec_list['IdList']
hdl = Entrez.efetch(db='nucleotide', id=id_list, rettype='gb', retmax=rec_list['Count'])

In [ ]:
print(id_list)

Let's now try to retrieve all these records. The following query will download all matching nucleotide sequences from GenBank, which are 281 at the time of
writing this book. You probably do not want to do this all the time

In [ ]:
recs = list(SeqIO.parse(hdl, 'gb'))

We will now just concentrate on a single record. This will only work if you used the exact same preceding query:

In [ ]:
for rec in recs:
    if rec.name == 'KM288867':
        break
print(rec.name)
print(rec.description)

Let's now extract some sequence features, which contain information such as gene products and exon positions on the sequence:

In [ ]:
for feature in rec.features:
    if feature.type == 'gene':
        print(feature.qualifiers['gene'])
    elif feature.type == 'exon':
        loc = feature.location
        print('Exon', loc.start, loc.end, loc.strand)
    else:
        print('not processed:\n%s' % feature)

We will now look at the annotations on the record, which is mostly metadata that is not related to the sequence position:

In [ ]:
for name, value in rec.annotations.items():
    print('%s=%s' % (name, value))

In [ ]:
print(len(rec.seq))

In [ ]:
print(rec.seq)

In [ ]:
refs = rec.annotations['references']
for ref in refs:
    if ref.pubmed_id != '':
        print(ref.pubmed_id)
        handle = Entrez.efetch(db="pubmed", id=[ref.pubmed_id],
                                rettype="medline", retmode="text")
        records = Medline.parse(handle)
        for med_rec in records:
            for k, v in med_rec.items():
                print('%s: %s' % (k, v))